In [1]:
%load_ext autoreload
%autoreload 2
import re
import numpy as np
import tensorflow as tf
import os
import sys
from tf2gpt.model import GPT
from utils.story_util import Story,Stories
from utils.progress_bar import ProgressBar
from tensorboardX import SummaryWriter
from tensorflow.keras.utils import multi_gpu_model
import random
import json

In [2]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [4]:
tf.keras.backend.set_floatx('float16')

In [5]:
#mirrored_strategy = tf.distribute.MirroredStrategy()

In [6]:
#with mirrored_strategy.scope():
gpt = GPT(
    vocab_size=30_000,
    layer_size=32,
    block_size=1024,
    embedding_dropout=0.0,
    embedding_size=2560,
    num_attention_heads=32,
    attention_dropout=0.0,
    residual_dropout=0.0,
    train_size=499
)
gpt.load_weights('./gpt_weight_pretrain/weight_fp16')

#input_x = tf.keras.layers.Input((499,), dtype=tf.int32)
#outputs = gpt_origin(input_x)

#gpt = tf.keras.Model(inputs=input_x, outputs=outputs)
#gpt = multi_gpu_model(gpt, gpus=8)

print(tf.keras.backend.floatx(), tf.float16, tf.keras.backend.floatx() == tf.float16)
if tf.keras.backend.floatx() == tf.float16:
    for x in gpt.weights:
        assert x.dtype == tf.float16


gpt.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),  # Optimizer
    # Loss function to minimize
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    # List of metrics to monitor
)

float16 <dtype: 'float16'> True


In [7]:
from utils.gpt2_tokenizer import GPT2Tokenizer
cbpe = GPT2Tokenizer(
    'CPM-Generate/bpe_3w_new/vocab.json',
    'CPM-Generate/bpe_3w_new/merges.txt',
    model_file='CPM-Generate/bpe_3w_new/chinese_vocab.model')

In [8]:
ids = cbpe.encode('今天天气还行')
ids

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.592 seconds.
Prefix dict has been built successfully.


[837, 259, 497, 57, 8, 237]

In [9]:
def test_basic_logic():
    ids = cbpe.encode('今天天气还行')
    #print(ids)
    print("+" * 20)
    for i in range(10):
        output = gpt(tf.constant([ids]))
        #print(output[0].shape)
        nid = np.argmax(output[0, -1])
        ids += [nid]
        #print(i, cbpe.decode(ids))
        #print(np.argmax(output[0],axis=-1))
        print(cbpe.decode(np.argmax(output[0],axis=-1)))
        print('-' * 30)
test_basic_logic()

++++++++++++++++++++
气 不错 不错行 
------------------------------
气 不错 不错行 ,
------------------------------
气 不错 不错行 , 我
------------------------------
气 不错 不错行 , 我 就
------------------------------
气 不错 不错行 , 我 就 想
------------------------------
气 不错 不错行 , 我 就 想着
------------------------------
气 不错 不错行 , 我 就 想着 去
------------------------------
气 不错 不错行 , 我 就 想着 去 看看
------------------------------
气 不错 不错行 , 我 就 想着 去 看看 
------------------------------
气 不错 不错行 , 我 就 想着 去 看看 ,
------------------------------


In [10]:
def get_learning_rate(learning_rate=6e-4,
                      warmup_steps=20_0000,
                      decay_steps=200_0000,
                      alpha=0.0):
    def decayed_learning_rate(step=1):
        if step <= warmup_steps:
            mult = step / float(warmup_steps)
        else:
            progress = (step - warmup_steps) / (decay_steps - warmup_steps)
            mult = 0.5 * (1 + math.cos(math.pi * progress))
            mult = max(0.1, mult)
        return learning_rate * mult
    return decayed_learning_rate

In [11]:
train_recoard = json.load(open('./labeled_data/advanture_translated/truncated_advanture_train.json'))
valid_recoard = json.load(open('./labeled_data/advanture_translated/truncated_advanture_valid.json'))

In [12]:
len(train_recoard),len(valid_recoard)

(4500, 266)

In [13]:
import copy
def data_generator(stories, batch_size=4,sample_len=200,inf=False):
    while True:
        batch_data = []
        tmp_stories = copy.copy(stories)
        random.shuffle(tmp_stories)
        for i,one_story in enumerate(tmp_stories):
            story_content = one_story.to_dungeon_format()
            story_content = story_content.replace("<start>\n","")
            story_content = story_content.replace("\n<end>","")
            story_content = story_content.replace("\n<end>","")
            story_content = story_content.replace(" ","")
            ids = cbpe.encode(story_content)
            while ids:
                sample = ids[:sample_len]
                ids = ids[sample_len:]
                if len(sample) < sample_len:
                    sample += [0 for i in range((sample_len - len(sample)))]
                batch_data.append(sample)
                if len(batch_data) >= batch_size:
                    yield np.asarray(batch_data)
                    batch_data = []
        if not inf:
            break
            
import copy
def data_generator_content(texts, batch_size=4,sample_len=200,inf=False):
    tmp_text = copy.copy(texts)
    segments = []
    while tmp_text:
        story_content = tmp_text[:10000]
        segments.append(story_content)
        tmp_text = tmp_text[10000:]
    while True:
        
        batch_data = []
        random.shuffle(segments)
        for one_story_context in segments:
            ids = cbpe.encode(one_story_context)
            while ids:
                sample = ids[:sample_len]
                ids = ids[sample_len:]
                if len(sample) < sample_len:
                    sample += [0 for i in range((sample_len - len(sample)))]
                batch_data.append(sample)
                if len(batch_data) >= batch_size:
                    yield np.asarray(batch_data)
                    batch_data = []
        if not inf:
            break
            
def turb_sentence(inputs):
    outputs = []
    for i in inputs:
        if random.random() > 0.1:
            outputs.append(i)
    return outputs
            
def data_generator_json(stories, batch_size=1,inf=False):
    while True:
        tmp_stories = copy.copy(stories)
        random.shuffle(tmp_stories)
        for i,one_story in enumerate(tmp_stories):
            previous_contents = one_story["previous"]
            previous_contents = turb_sentence(previous_contents)
            action = one_story["action"]
            content = one_story["result"]
            
            previous_contents = ''.join(previous_contents)
            content = ''.join(content)
            
            ids_prev = cbpe.encode(previous_contents + "\n")
            ids_action = cbpe.encode(">" + action + "\n")
            ids_content = cbpe.encode(content)[:300]
            
            ids_prev = ids_prev + ids_action
            
            prev_context_len = int(random.uniform(100,500))
            ids_prev = ids_prev[-prev_context_len:]
            
            overall_ids = ids_prev + ids_content
            mask = [0 for i in ids_prev] + [1 for i in ids_content]
            yield np.asarray([overall_ids]),np.asarray([mask])
            
        if not inf:
            break

In [14]:
valid_gen = data_generator_json(valid_recoard,inf=True)

In [15]:
x,m = valid_gen.__next__()

In [16]:
x.shape,m.shape

((1, 687), (1, 687))

In [17]:
cbpe.decode(x[0])

"。 当 里德 看到 他 的 妻子 玛格丽特 和 他们 的 几个 孩子 时 , 他 高兴 地 叫 了 一声 , 冲 上前 拥抱 他们 。 你 很快 就 了解 到 一些 非常 令人担忧 的 事情 : 在 山上 的 营地 , 唐纳 党 的 其余 成员 正因 饥饿 而 不断 死亡 , 刚 从 山地车 司机 约翰 丹顿 和 孩子 艾达凯 塞 伯格 下山 , 在 第一次 救济 旅行 时 死亡 .   19   名 幸存者 留在 了 第一次 救援 , 但 更 多 的 人 仍然 在 山上 , 包括 里德 的 两个 孩子 。 威廉 · 麦卡琴 得知 他 年幼 的 女儿 哈丽特 在 山上 去世 了 , 他 很 难 接受 这个 消息 。 当 您 在 冰雪 覆盖 的 荒原 上 月 的 第一天 到达 客舱 的 乱堆 的 第一 人 , 你 相遇 是 里德 “ 的 女儿 帕蒂 , 对 她 的 父亲 , 他们 用 欢呼 迎接 冲刺 。 其他 一些 人影 从小 屋里 出来 , 包括 帕特里克 · 布林 和 路易斯 · 凯塞 伯格 , 总共 有   19   人 仍然 住 在 特拉 基湖 周围 的 小 屋里 , 大约 有   12   人住 在 奥尔德 溪 , 包括 乔治 唐纳 。 只 呆 一天 , 你 就 聚集 了 十七名 幸存者 , 你 可以 把 所有 的 人 都 带走 。 进行 了 一些 讨论 , 并 同意 留下 一两个 人来 帮助 留下来 的   13   个人 , 尤其 是 在   Alder   Creek   的 那些 人 。 你 做 什么 工作 ? \n > 你 带 着 第二次 救济 的 其余部分 离开 \n 你 选择 陪 在 第二 浮雕 和 17 个 定居 的 艰难 跋涉 回到 萨特 ' 堡垒 。 当 一场 风暴 将 你们 所有人 困在 开阔 的 地面 上 , 艾萨克 · 唐纳   ( Isaac   Donner )   去世 时 , 事情 的 开端 并 不 顺利 。 James   Reed   和   William   McCutchen   决定 , 因为 供应量 太 少 , 他们 必须 继续前进 , 但 大多数 定居者 太 虚弱 而 无法 移动 。 带 着 里德 的 两个 孩子 和 第三个 孩子 继续前进 , 无法 和 其他人 呆 在 一起 。 几天 后 , 您 遇到 

In [18]:
m

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [19]:
writer = SummaryWriter('log/finetune')
n_iter = 0
train_loss = 100
val_loss = 100
for epoch in range(100):
    print(f"Epoch {epoch + 1}")
    pb = ProgressBar(len(train_recoard))
    pb.startjob()
    for x,m in data_generator_json(train_recoard):
    #for x in data_generator_content(texts[528:],sample_len=500,batch_size=2):
        n_iter += 1
        ret = gpt.train_step(
            (
                tf.constant(x[:,:-1]),
                tf.constant(x[:,1:])
            ),
            mask=m[:,1:],
        )
        writer.add_scalar("train/loss", ret["loss"].numpy(), n_iter)
        train_loss = ret["loss"].numpy()
        if n_iter % 10 == 0:
            valid_x,valid_m = valid_gen.__next__()
            ret = gpt.eval_step(
            (
                tf.constant(valid_x[:,:-1]),
                tf.constant(valid_x[:,1:])),
                mask=valid_m[:,1:],
            )
            writer.add_scalar("test/loss", ret["loss"].numpy(), n_iter)
            val_loss = ret["loss"].numpy()
        pb.info = f"tl: {train_loss} vl: {val_loss}"
        pb.complete(1)
    print()

Epoch 1
tl: 2.453125 vl: 2.912109375 35.07 % [=================>---------------------------------] 1578/4500 	 used:1748s eta:3237 s9 s s

/home/tiger/.local/lib/python3.7/site-packages/numpy/core/fromnumeric.py:87: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


tl: 2.486328125 vl: 3.01171875 100.00 % [==================================================>] 4500/4500 	 used:4933s eta:0 sssss17 s
Epoch 2
tl: 2.33984375 vl: 2.556640625 97.20 % [================================================>--] 4374/4500 	 used:4759s eta:137 sssss8 sWarning: NaN or Inf found in input tensor.
tl: 2.974609375 vl: 2.814453125 100.00 % [==================================================>] 4500/4500 	 used:4897s eta:0 ss
Epoch 3
tl: 2.671875 vl: 1.8720703125 44.38 % [======================>----------------------------] 1997/4500 	 used:2171s eta:2721 s sssWarning: NaN or Inf found in input tensor.
tl: 2.12109375 vl: 2.6640625 100.00 % [==================================================>] 4500/4500 	 used:4893s eta:0 s s ssss
Epoch 4
tl: 3.0703125 vl: 2.626953125 9.42 % [====>----------------------------------------------] 424/4500 	 used:460s eta:4424 s ssWarning: NaN or Inf found in input tensor.
tl: 2.28515625 vl: 2.62890625 100.00 % [===============================

KeyboardInterrupt: 

In [ ]:
m.shape,x.shape

In [ ]:
print(stories[0].to_dungeon_format())

# 使用

In [23]:
q = f'''你是一个公司老总，你事业正在上升期，你娶了一个美丽的妻子
> 你走进你的家门
'''
ids = cbpe.encode(q)
#print(ids)
#print("+" * 20)
for i in range(100):
    output = gpt(tf.constant([ids]))
    nid = np.argmax(output[0, -1])
    ids += [nid]
    print(i)
    
print(i,cbpe.decode(ids))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
99 你 是 一个 公司老总 , 你 事业 正在 上升期 , 你 娶 了 一个 美丽 的 妻子 
 >   你 走进 你 的 家门 
 你 是 一个 公司老总 , 你 事业 正在 上升期 , 你 娶 了 一个 美丽 的 妻子 
 >   
 你 的 妻子 回来 了 
 >   
 你 的 妻子 回来 了 
 >   
 你 的 父母 了 
 >   
 你 的 父母 了 
 
 >   
 你 的 父母 了 
 
 >   
 他们


In [20]:
gpt.save_weights('./gpt_weight_pretrain/tmp_weight')

# save model (restart kernel here)

In [1]:
%load_ext autoreload
%autoreload 2
import re
import numpy as np
import tensorflow as tf
import os
import sys
from tf2gpt.model import GPT
from utils.story_util import Story,Stories
from utils.progress_bar import ProgressBar
from tensorboardX import SummaryWriter
from tensorflow.keras.utils import multi_gpu_model
import random
tf.keras.backend.set_floatx('float16')

In [2]:
from utils.gpt2_tokenizer import GPT2Tokenizer
cbpe = GPT2Tokenizer(
    'CPM-Generate/bpe_3w_new/vocab.json',
    'CPM-Generate/bpe_3w_new/merges.txt',
    model_file='CPM-Generate/bpe_3w_new/chinese_vocab.model')

In [3]:
#with mirrored_strategy.scope():
gpt = GPT(
    vocab_size=30_000,
    layer_size=32,
    block_size=1024,
    embedding_dropout=0.0,
    embedding_size=2560,
    num_attention_heads=32,
    attention_dropout=0.0,
    residual_dropout=0.0
)
gpt.load_weights('./gpt_weight_pretrain/tmp_weight')

#input_x = tf.keras.layers.Input((499,), dtype=tf.int32)
#outputs = gpt_origin(input_x)

#gpt = tf.keras.Model(inputs=input_x, outputs=outputs)
#gpt = multi_gpu_model(gpt, gpus=8)

print(tf.keras.backend.floatx(), tf.float16, tf.keras.backend.floatx() == tf.float16)
if tf.keras.backend.floatx() == tf.float16:
    for x in gpt.weights:
        assert x.dtype == tf.float16


gpt.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),  # Optimizer
    # Loss function to minimize
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    # List of metrics to monitor
)

float16 <dtype: 'float16'> True


In [4]:
ids = cbpe.encode('今天天气不错')

for i in range(10):
    output = gpt(tf.constant([ids]))
    nid = np.argmax(output[0, -1])
    ids += [nid]
    print(i, cbpe.decode(ids))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.563 seconds.
Prefix dict has been built successfully.


0 今天天气 不错 
1 今天天气 不错 ,
2 今天天气 不错 , 不如
3 今天天气 不错 , 不如 出去
4 今天天气 不错 , 不如 出去 走走
5 今天天气 不错 , 不如 出去 走走 吧
6 今天天气 不错 , 不如 出去 走走 吧 
7 今天天气 不错 , 不如 出去 走走 吧 。
8 今天天气 不错 , 不如 出去 走走 吧 。 
9 今天天气 不错 , 不如 出去 走走 吧 。 ”


In [5]:
def batch_gather(a, b):
    return tf.gather(a, b, batch_dims=1)

def penalize_used(logits, output,penalize):
    # I want to change the indices of logits wherever the index is found in output
    change_tensor = tf.zeros_like(logits, dtype=logits.dtype)
    unique = tf.unique(output[0])[0]
    ones = tf.ones_like(unique, dtype=unique.dtype)
    indices = tf.expand_dims(unique, 1)
    updates = tf.scatter_nd(indices, ones, [logits.shape[1]])
    bool_tensor = tf.expand_dims(tf.cast(updates, tf.bool), 0)
    return tf.compat.v1.where(bool_tensor, logits * penalize, logits)

def top_p_sample(logits, inputs, num_samples=1, p=0.9,k=40,temperature=0.4,penalize=0.4):
    logits = penalize_used(logits,inputs,penalize)
    batch_size, vocab_size = logits.shape
    probs = tf.nn.softmax(logits / temperature, axis=-1)
    # [batch_size, vocab_perm]
    indices = tf.argsort(probs, direction='DESCENDING')
    cumulative_probabilities = tf.math.cumsum(batch_gather(probs, indices), axis=-1, exclusive=False)
    top_k_prob = cumulative_probabilities[0][k]

    # find the top pth index to cut off. careful we don't want to cutoff everything!
    # result will be [batch_size, vocab_perm]
    p_expanded = p if isinstance(p, float) else p#[:, None]
    exclude_mask = tf.logical_not(
        tf.logical_or(
            tf.logical_and(
                cumulative_probabilities < p_expanded, 
                cumulative_probabilities < top_k_prob
            ),
            tf.range(vocab_size)[None] < 1
        )
    )

    # OPTION A - sample in the sorted space, then unsort.
    logits_to_use = batch_gather(logits, indices) - tf.cast(exclude_mask, tf.float16) * 1e4
    sample_perm = tf.random.categorical(logits=logits_to_use, num_samples=num_samples)
    sample = batch_gather(indices, sample_perm)

    return tf.cast(sample, tf.int64)

In [6]:
@tf.function
def serve(inputs):
    return gpt(inputs, kv_cache=None, use_cache=True)


@tf.function
def serve_cache(inputs, kv_cache):
    return gpt(inputs, kv_cache=kv_cache, use_cache=True)

In [7]:
serve_concrete = serve.get_concrete_function(
    tf.TensorSpec(shape=[None, None], dtype=tf.int64, name="inp")
)

layer_size = 32
attention_head = 32
embedding_size = 2560

serve_cache_concrete = serve_cache.get_concrete_function(
    tf.TensorSpec(shape=[None, None], dtype=tf.int64, name="inp"),
    tf.TensorSpec(shape=[
        layer_size, None, 2, attention_head,
        None, embedding_size // attention_head
    ], dtype=tf.float16, name="kv_cache")
)

In [8]:
r = serve_concrete(
    tf.constant([[1]], tf.int64)
)
r2 = serve_cache_concrete(
    tf.constant([[1]], tf.int64),
    r[1]
)

In [9]:
@tf.function
def sample(initial_inputs, length, p,k,temperature,penalize, num_samples=1):
    layer_size = 32
    embedding_size = 2560
    attention_head = 32

    i = tf.constant(0, dtype=tf.int64)
    initial_logits, kv_cache = serve(initial_inputs)
    inputs = top_p_sample(initial_logits[:, -1, :],initial_inputs, p=p,k=k,temperature=temperature,penalize=penalize)
    stores = tf.concat([initial_inputs, inputs], axis=1)

    def _cond(i, inputs, kv_cache, stores):
        return i < length

    def _body(i, inputs, kv_cache, stores):
        new_logits, new_kv_cache = serve_cache(inputs, kv_cache)
        
        new_inputs = top_p_sample(new_logits[:, -1, :],inputs, p=p,k=k,temperature=temperature,penalize=penalize)
        new_stores = tf.concat([stores, new_inputs], axis=-1)
        new_kv_cache = tf.concat([
            kv_cache,
            new_kv_cache
        ], axis=-2)
        new_i = i + 1
        return [new_i, new_inputs, new_kv_cache, new_stores]

    result = tf.while_loop(
        _cond, _body,
        loop_vars=[i, inputs, kv_cache, stores],
        shape_invariants=[
            tf.TensorShape(None),
            tf.TensorShape([None, None]),
            tf.TensorShape([
                layer_size, None, 2,
                attention_head, None,
                embedding_size // attention_head
            ]),
            tf.TensorShape([
                None, None
            ])
        ]
    )
    return result[-1]

In [10]:
gpt.save('./result_weights/tmp_weight', include_optimizer=False, signatures={
    'serving_default': sample.get_concrete_function(
        tf.TensorSpec(shape=[None, None], dtype=tf.int64, name="inp"),
        tf.TensorSpec(shape=[None,], dtype=tf.int64, name="length"),
        tf.TensorSpec(shape=[], dtype=tf.float16, name="p"),
        tf.TensorSpec(shape=[], dtype=tf.int64, name="k"),
        tf.TensorSpec(shape=[], dtype=tf.float16, name="temperature"),
        tf.TensorSpec(shape=[], dtype=tf.float16, name="penalize")
    )
})

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ./result_weights/tmp_weight/assets


In [11]:
ids = cbpe.encode('今天天气不错')

ret = sample(
    tf.constant([ids], dtype=tf.int64), # inp
    tf.constant(10, dtype=tf.int64), #length
    tf.constant(0.9, dtype=tf.float16), #p
    tf.constant(40, dtype=tf.int64), #k
    tf.constant(0.4, dtype=tf.float16), # temperature
    tf.constant(0.4, dtype=tf.float16) # penalize
)
print(ret)
print(cbpe.decode(ret.numpy().tolist()[0]))

tf.Tensor(
[[  837   259   497   788     8     9  4609    64  1555 14981     8    12
      8    34     8]], shape=(1, 15), dtype=int64)
今天天气 不错 , 正好 可以 出去 走走 。 ” 


In [12]:
!ls -l ./result_weights/tmp_weight

total 23564
drwxr-xr-x 2 tiger tiger     4096 Jul  5 15:21 assets
-rw-r--r-- 1 tiger tiger 24117909 Jul 12 10:34 saved_model.pb
drwxr-xr-x 3 tiger tiger     4096 Jul 12 10:34 variables


# read deploy model (you should restart kernel and go from here)

In [2]:
%load_ext autoreload
%autoreload 2
import re
import numpy as np
import tensorflow as tf
import os
import sys
from tf2gpt.model import GPT
from utils.story_util import Story,Stories
from utils.progress_bar import ProgressBar
from tensorboardX import SummaryWriter
from tensorflow.keras.utils import multi_gpu_model
import random
from utils.gpt2_tokenizer import GPT2Tokenizer
import tensorflow_hub as hub
cbpe = GPT2Tokenizer(
    'CPM-Generate/bpe_3w_new/vocab.json',
    'CPM-Generate/bpe_3w_new/merges.txt',
    model_file='CPM-Generate/bpe_3w_new/chinese_vocab.model')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
gpt = hub.load('./result_weights/tmp_weight/')

In [7]:
def sample_gpt(tokenizer, gpt, sentence, number=1, length=20, p=0.9,k=40,temperature=0.4,penalize=0.85):
    inputs = tf.constant([tokenizer.encode(sentence)] * number, dtype=tf.int64)
    length = tf.constant(length, dtype=tf.int64)
    p = tf.constant(p, dtype=tf.float16)
    k = tf.constant(k, dtype=tf.int64)
    temperature = tf.constant(temperature, dtype=tf.float16)
    penalize = tf.constant(penalize, dtype=tf.float16)
    ret = gpt.signatures['serving_default'](
        inp=inputs, 
        length=length, 
        p=p,
        k=k,
        temperature=temperature,
        penalize=penalize)['output_0']
    return [
        tokenizer.decode(s[len(inputs[0]):]).replace(' ', '')
        for s in ret.numpy()
    ]

In [29]:
class Story():
    def __init__(self,beginning,story_max_len=200,context_len=12):
        self.story = [beginning]
        self.story_max_len = story_max_len
        self.context_len = context_len
        
    def response_quality_ok(self,response):
        if len(response) < 20:
            return False
        return True
    
    def action(self,action):
        action_str = "> 你" + action
        q = "".join(self.story[-self.context_len:]) + "\n" + action_str + "\n"
        self.story.append("你" + action)
        response = ""
        for i in range(3):
            response = sample_gpt(cbpe, gpt, q, 1, 150,p=0.9,k=16,temperature=1,penalize=0.75)[0]
            response = '。'.join(response.split("。")[:-1]) + "。"
            if(self.response_quality_ok(response) == True):
                break
            else:
                print("response quality check failed,generating another one")
        
        #responsesp = response.split("。")
        #if(len(responsesp) > 2):
        #    response = ''.join(responsesp[:-1])
        #else:
        #    response = response.split(">")[0]
        #response = "。".join([:-1])
        #response = response.split(">")[0].strip()
        #response = "\n".join(response.split("\n")[:-1])
        self.story.append(response)
        
    def interactive(self):
        print("\n".join(self.story))
        while True:
            action = input("> 你")
            self.action(action)
            print(self.story[-1])

In [36]:
begin = f'''你在树林里冒险，指不定会从哪里蹦出来一些奇怪的东西，你握紧手上的手枪，希望这次冒险能够找到一些值钱的东西'''
story = Story(begin)

In [ ]:
story.interactive()

你在树林里冒险，指不定会从哪里蹦出来一些奇怪的东西，你握紧手上的手枪，希望这次冒险能够找到一些值钱的东西


> 你 前往一个城镇


你已经厌倦了这个地方,你现在需要尽快逃离这里,而这会为你的安全提供一些帮助。你沿着街道一直走,偶尔看到一两个人在街道上移动,你也没太在意,他们的行动似乎表明他们在等待。“你怎么知道,你现在看到了什么?”你问一个人,这个人正在抱怨他的同伴。“如果他们都在树林里等待着什么,那他们为什么要在这里等着我?”你问。


> 你 走进一家酒馆


当你进入酒馆的时候,你立刻开始感到不安,因为酒馆里有人在交谈,你很快就会被淹没在他们的谈话中。“这里有没有一个,”你喊道,但在你说“不是”之前,“他们都不,”一个人回答,“他们是在谈论昨天晚上发生的事情,我看到他们在外面的人。”“好吧,那是什么?”你问他,他看起来似乎很惊讶。


...

睡眠是一种享受,而现在睡眠是多么糟糕的事情。你感觉自己在发烧,你看到了你的身体上所有的痛苦,当你终于感觉到疼痛时,你的头很疼,你在这里的每一个晚上都在发烧,你希望能让自己睡觉,你希望在第二天早上醒来,但还是让自己入睡。你希望睡觉可以让你的头痛减少,让你能在睡眠中继续做梦。你希望睡眠的时候不要被噩梦惊醒,你希望睡眠的时候不会有噩梦的惊醒。


> 你 下楼吃点感冒药


当你下楼吃感冒药时,你感觉不到任何冰冷的感觉,当你下楼时,你感觉很温暖,感觉不到任何恐惧。你终于吃完了感冒药,你感到胃里好像有东西在颤抖,你开始感觉到寒冷,但你仍然感觉很好。你的喉咙似乎被堵住了,但你仍然在打喷嚏,让你感觉很好。你的喉咙开始灼热,当你感觉有点冷的时候,它已经是你唯一的感觉。


> 你 走出门观察小镇


早晨在街道上游荡是很正常的,因为你只是想找到一些人,然后把所有人都吓一跳,你希望在任何人离开的时候都是这样。你在街上走着,你可以感觉到有什么东西在你的身上起起伏伏,感觉就像有东西在你的身体里一样。你走进酒馆当你进入酒馆的时候,你立刻开始感到不安,因为酒馆里有人在交谈,你很快就会被淹没在他们的交谈中。
